In [1]:
#######################
#basic standard modules
#######################
import sys, os
import time
import collections, itertools, copy, operator

#######################
#custom config modules
#######################
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
load_dotenv()

#######################
#file manipulation modules
#######################
import pickle, json

#######################
#string manipulation modules
#######################
import re, string
import nltk
print("in the other",os.getenv("TEST_MULTILINE_VAR"))
nltk.data.path.append(os.getenv("NLTKDATADIR"))
from bs4 import BeautifulSoup

#######################
#general data manipulation and data analysis modules
#######################
import pandas, gensim, sklearn, scipy, numpy, math

#######################
#custom modules
#######################
import processingData

in the other hello\nworld


Using TensorFlow backend.


in the other hello\nworld


In [36]:
############################
## extract_candidate_chunks: candidate phrases based on http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/
############################
def extract_candidate_chunks(lemmpostxt, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}', STOPWORDS = nltk.corpus.stopwords.words('english')):
    #def redotaggedtext(lemmpostxt, lemmpossts = [], lemmposst = []):
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    lemmpostxt_redo = []
    for lemmposst in lemmpostxt:
        lemmposst_redo = []
        for w,val,l,pos in lemmposst:
            if w not in STOPWORDS:
                #or not all(char in punct for char in w):
                lemmposst_redo.append((l,pos))
        lemmpostxt_redo.append(lemmposst_redo)
            
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(lemmposst_redo)) for lemmposst_redo in lemmpostxt_redo))

    #print(all_chunks)
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group)
                  for key, group in itertools.groupby(all_chunks, lambda w_TUPLE: w_TUPLE[2] != 'O') if key]

    return [cand for cand in candidates]

In [37]:
############################
## allrecordsPreparation 3: revision of 2 to extend it for keyphrase candidate analysis
## some articles:
## -- https://stackoverflow.com/questions/34714162/preventing-splitting-at-apostrophies-when-tokenizing-words-using-nltk
############################

def allrecordsPreparation3(allrecords, STOPWORDS=nltk.corpus.stopwords.words('english'), punct = set(string.punctuation)):
    '''
    description: tokenization and POS tagging
    input: dict of allrecords texts and data from different sources
    treatment: separating only those with posts in the forum and tokenizing the posts
    output:
    1) list of lists, each with:
    -- id
    -- username
    -- link of the post
    -- tokenized text
    -- POS tagged text
    2) list of post ids
    '''
    print('in allrecordsPreparation (len(allrecords))::',len(allrecords))
 
       
    def HTMLtoText(u):
        #block arguments
        record = u["data"]
        forumpost = record['forum']['foundjob_msg']['text']
        if forumpost == '':
            return False
        forumpostID = record['forum']['foundjob_msg']['id']
        forumpostLINK = record['forum']['foundjob_msg']['link']
        soup_forumpost = BeautifulSoup(forumpost)
        soup_forumpostTEXT = soup_forumpost.find('body').get_text().replace('’',"'")
        tksoup_forumpostTEXT = [
                                #nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) 
                                token.lower()
                                for token in nltk.word_tokenize(soup_forumpostTEXT)
                                #if token.lower() not in STOPWORDS 
                                #and not re.match(r'\d+?', token)
                                #and not all(char in set(string.punctuation) for char in token)
                               ]
        return forumpostID, forumpostLINK, tksoup_forumpostTEXT
    
    def approxsts(text):
        modtext = []
        for w in tksoup_forumpostTEXT:
            w = w.lower()
            rws = []
            if len(w) > 1 and len({'.','-',':'}.intersection(w)) >= 1:
                #print(w)
                for punc in {'.','-',':'}.intersection(w):
                    rws = w.replace(punc, ' '+punc+' ').split()
                #print(rws)
            if len(rws) == 0:
                modtext.append(w)
            else:
                for w in rws:
                    modtext.append(w)
        return modtext
    
    def lemmatizationofpos(postxt, lemmws):
        lemmposws = []
        counterrors = 0
        countKerrors = 0
        countIerrors = 0
        countNCerrors = 0
        for posw in postxt:
            w = posw[0]
            pos = posw[1]
            if nltk.corpus.wordnet.synsets(w):
                try:
                    n = ''
                    if nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0] == 'entity':
                        n = w
                    else:
                        n = nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0]
                    if pos[0] == 'V':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w,'v')
                    if pos == 'NNS' or pos == 'NN$':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w)
                    #print(w, nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms(), n)
                    lemmposws.append((w,True,n,pos))
                    lemmws.append(n)
                except KeyError: #in some cases the POS tag is not recognised by wordnet synset
                    print("pos KeyErrors", w,pos)
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countKerrors += 1
                except IndexError:
                    print("IndexErrors (POS not found)", w,pos) #in some cases (w,pos) pair was not found at some point of the synsets root hyernyms
                    #print(nltk.corpus.wordnet.synsets(w,pos[0].lower()))
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countIerrors += 1
            else:
                #print(w, [])
                print("Error (not in corpus)", w,pos)
                lemmposws.append((w,False,w,pos))
                lemmws.append(w)
                counterrors += 1
                countNCerrors += 1
        #print("\n\ntotal number of errors : ", counterrors)
        #print("total number of noPOSerrors : ", countKerrors)
        #print("total number of indexerrors : ", countIerrors)
        #print("total number of nonincorpuserrors : ", countNCerrors)
        return lemmposws, counterrors
      
    
    lemmws = []
    lemmposrecs = []
    count = 0
    lemerrors = 0
    for u in allrecords:
        ## Getting the data as a text from HTML format (raw dataset)
        userdata = HTMLtoText(u)
        if userdata != False:
            forumpostID, forumpostLINK, tksoup_forumpostTEXT = userdata
            
            ## Sentence identification, tokenization and POS
            txt2possts = []
            st = []
            for token in tksoup_forumpostTEXT:
                st.append(token)
                if re.match(r'^[.!?]+|\n$', token):
                    postst = nltk.pos_tag(st)
                    txt2possts.append(postst)
                    st = []
            if not re.match(r'^[.!?]+|\n$', token):
                posst = nltk.pos_tag(st)
                txt2possts.append(posst)

           
            ## Lemm text
            lemmpostxt = []
            for posst in txt2possts:
                lemst, err = lemmatizationofpos(posst, lemmws)
                lemmpostxt.append(lemst)
                lemerrors += err

            
            ## Keyphrases candidates; complete lemmws with candidates that are not still there
            candidates = extract_candidate_chunks(lemmpostxt)
            for cand in candidates:
                if cand not in lemmws:
                    lemmws.append(cand)
            
            ## Adding data to the new created dataset
            lemmposrecs.append((
                            'f_'+forumpostID,
                            u["user"],
                            forumpostLINK,
                            lemmpostxt,
                            candidates
                            ))
            count += 1

    print("number of treated posts (len(count)) ::", count)
    print("lemm errs:", lemerrors)
    #return all_posedsts, forum_ids
    return lemmposrecs, nltk.FreqDist(lemmws)

In [3]:
############################
## wordimportance_var4: wordimportance_var3 modified to fit keyphrases; changes in the "opacity" formula
############################
def wordimportance_var4(lemmposrecs, lemmws_fd):
    '''
    description:
    
    This metric tries to solve some of the issues that appeared in `wordimportance_var1` metric, in particular the values of zero.
    
    This is done by setting lower bounds when required.
    
    --- In the case of opacity, a non-zero lower bound is set by changing the equation to the following:
        ```
        if 1-math.log(v)/maxdiv == 0: 1-math.log(maxdiv-1)/maxdiv # 1-math.log(v)/maxdiv == 0 if v == maxdiv
        ```
    --- In the case of sizing, a redefinition of the metric force a non-zero lower bound as well as rebumpimg rare terms in documents:
        ```
        (sum(vector)-max(vector))/sum(vector)
        ```
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance 
    '''
    unigrams = dict([(grams, count) 
                     for grams, count in lemmws_fd.items() 
                     if len(grams.split()) == 1 or (len(grams.split()) == 2 and '' in grams.split())
                    ])
    maxdiv = math.log(sorted(unigrams.items(), key=lambda x: x[1], reverse=True)[0][1])
    opacity = collections.defaultdict(float)
    for grams, counts in lemmws_fd.items():
        opval = []
        for gram in grams.split():
            if gram == '':
                continue
            if gram in unigrams:
                opval.append(math.log(unigrams[gram]))
            else:
                opval.append(0)
        averopval = sum(opval)/len(opval)
        if 1 - averopval/maxdiv != 0.0:
            opacity[grams] = 1 - averopval/maxdiv
        else:
            opacity[grams] = 1-math.log(maxdiv-1)/maxdiv
        
    sizing_matrix = dict([(w, [0]*len(lemmposrecs)) for w in list(lemmws_fd.keys())])

    ## Count lemmatized words/characters per text  
    for i,lemmpos_t in enumerate(lemmposrecs):
        for k, lemmpos_TUPLE in enumerate(lemmpos_t[3]):
            ## Use lemmatized word
            lemmw = lemmpos_TUPLE[2]
            sizing_matrix[lemmw][i] = sizing_matrix[lemmw][i] + 1
        for cand in lemmpos_t[4]: #<--------------------------------- problem!! it counted candidates only once!!!
            if cand not in sizing_matrix:
                sizing_matrix[cand][i] = sizing_matrix[cand][i] + 1

       
    ## Normalization
    normalization = dict([(k, (sum(vector)-max(vector))/sum(vector)) if sum(vector) != 0 else (k,0.0) for k, vector in sizing_matrix.items()])
    #normalization = collections.defaultdict(float)
    #for k, vector in sizing_matrix.items():
    #    if sum(vector) != 0:
    #        normalization[k] = (sum(vector)-max(vector))/sum(vector)
    #    else:
    #        
        
    
    wordimportance = dict([(k, valnorm*opacity[k]) for k, valnorm in normalization.items()])

    return wordimportance

In [4]:
def cleaningtext(st, STOPWORDS = nltk.corpus.stopwords.words('english')):
    treated_st = []
    countwds = len(st)
    for w in st:
        if 'freecodecamp' in w:
            w = w.replace('freecodecamp','fcc')
        treated_st.append(w)
    return treated_st, countwds

In [16]:
############################
## gensim_models2 re-evaluated: modification of gensim_model2's cleanedsts_from_lemmpostxts to fit keyphrases
############################

def gensim_models2(lemmposrecs, NUM_TOPICS = 15, lemmws_fd = {}, wordimportance = {}, nltk = nltk, gensim = gensim):
    
    def cleanedsts_from_lemmpostxts2(lemmposrecs, STOPWORDS = nltk.corpus.stopwords.words('english')):
        redo_corpus_by_sts = []
        for lemmpos_r in lemmposrecs:
            candidates = lemmpos_r[4]
            treated_lemmcands = cleaningtext(candidates)[0] #passing candidates only
            redo_corpus_by_sts.append(treated_lemmcands) 
        
        #print(len(redo_corpus_by_sts), redo_corpus_by_sts[-1])
        return redo_corpus_by_sts
        
    def basedonBOW(redo_corpus_by_sts):
        dictionary = gensim.corpora.Dictionary(redo_corpus_by_sts) #[token for st in redo_corpus_by_sts for token in st]
        corpus = [dictionary.doc2bow(text) for text in redo_corpus_by_sts]
        return corpus, dictionary
    
    def basedonTFIDF(corpus):
        return gensim.models.TfidfModel(corpus)
    
    def basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance):
        
        def metriccalc(w):
            if w in wordimportance:
                return 1.0+2.0**float(wordimportance[w])
            else:
                return 0.0

        corpus = []
        for sts in redo_corpus_by_sts:
            st = []
            for w in sts:
                st.append((dictionary.token2id[w], metriccalc(w)))
            corpus.append(st)
        return corpus
        
    
    redo_corpus_by_sts  = cleanedsts_from_lemmpostxts2(lemmposrecs)
    
    corpus, dictionary = basedonBOW(redo_corpus_by_sts)
    if wordimportance == {'tfidf':True}:
        tfidf = basedonTFIDF(corpus)
        corpus = tfidf[corpus]
    if wordimportance != {} and wordimportance != {'tfidf':True}:
        corpus = basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance)
    
    
    lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, minimum_probability=0.005, per_word_topics = True, minimum_phi_value = 0.001, id2word=dictionary)
    lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

   
    return lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary

In [6]:
with open('../data/jobproject_forum.json','r') as message:
    otp = json.load(message)
print(len(otp))
allrecords = [{ "user": k, "data": otp[k] }  for k in otp]
print(len(allrecords))

90
90


In [38]:
lemmposrecs, lemmws_fd = allrecordsPreparation3(allrecords)

in allrecordsPreparation (len(allrecords)):: 90
Error (not in corpus) ! .
pos KeyErrors happy JJ
Error (not in corpus) to TO
Error (not in corpus) , ,
Error (not in corpus) that IN
Error (not in corpus) my PRP$
pos KeyErrors first JJ
Error (not in corpus) offer/trainee JJ
Error (not in corpus) . .
Error (not in corpus) the DT
pos KeyErrors a DT
pos KeyErrors digital JJ
Error (not in corpus) and CC
Error (not in corpus) . .
Error (not in corpus) they PRP
Error (not in corpus) , ,
pos KeyErrors graphic JJ
Error (not in corpus) and CC
pos KeyErrors some DT
pos KeyErrors basic JJ
Error (not in corpus) . .
Error (not in corpus) my PRP$
pos KeyErrors will MD
pos KeyErrors graphic JJ
Error (not in corpus) and CC
pos KeyErrors basic JJ
Error (not in corpus) ( (
Error (not in corpus) html5 NN
Error (not in corpus) , ,
Error (not in corpus) , ,
Error (not in corpus) ) )
Error (not in corpus) . .
pos KeyErrors a DT
Error (not in corpus) of IN
pos KeyErrors a DT
pos KeyErrors paid JJ
Error (not in

/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


pos KeyErrors a DT
pos KeyErrors 21 CD
pos KeyErrors old JJ
pos KeyErrors in IN
Error (not in corpus) the DT
pos KeyErrors united JJ
Error (not in corpus) with IN
pos KeyErrors no DT
Error (not in corpus) . .
pos KeyErrors a DT
Error (not in corpus) and CC
pos KeyErrors a DT
pos KeyErrors ago IN
pos KeyErrors i JJ
Error (not in corpus) this DT
Error (not in corpus) “how NN
pos KeyErrors 2 CD
Error (not in corpus) of IN
Error (not in corpus) my PRP$
Error (not in corpus) to TO
pos KeyErrors before IN
pos KeyErrors free JJ
Error (not in corpus) camp” NN
Error (not in corpus) . .
pos KeyErrors a DT
Error (not in corpus) and CC
pos KeyErrors a DT
IndexErrors (POS not found) i RB
Error (not in corpus) my PRP$
Error (not in corpus) of IN
pos KeyErrors all PDT
Error (not in corpus) the DT
pos KeyErrors free JJ
Error (not in corpus) and CC
pos KeyErrors on IN
Error (not in corpus) of IN
Error (not in corpus) that DT
pos KeyErrors a DT
Error (not in corpus) front-end JJ
Error (not in corpus) . 

In [13]:
wordimportance = wordimportance_var4(lemmposrecs, lemmws_fd)

In [26]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, wordimportance = wordimportance)

NameError: name 'wordimportance' is not defined

In [15]:
lda_model.print_topics()

[(0,
  '0.005*"lot" + 0.005*"interview" + 0.005*"company" + 0.005*"year" + 0.005*"month" + 0.005*"github" + 0.005*"day" + 0.005*"fcc" + 0.004*"experience" + 0.004*"job"'),
 (1,
  '0.007*"something" + 0.006*"day" + 0.005*"people" + 0.004*"mistake" + 0.004*"interview" + 0.004*"time" + 0.004*"project" + 0.004*"month" + 0.004*"portfolio" + 0.004*"company"'),
 (2,
  '0.007*"job" + 0.004*"company" + 0.004*"project" + 0.004*"year" + 0.004*"code" + 0.004*"lot" + 0.003*"group" + 0.003*"question" + 0.003*"someone" + 0.003*"people"'),
 (3,
  '0.009*"company" + 0.005*"bit" + 0.005*"project" + 0.004*"anything" + 0.004*"something" + 0.003*"money" + 0.003*"full time job" + 0.003*"stuff" + 0.003*"it’s" + 0.003*"huge thank quincy everyone community"'),
 (4,
  '0.005*"experience" + 0.004*"share" + 0.004*"job" + 0.004*"solve challenge" + 0.003*"app" + 0.003*"interview" + 0.003*"thing" + 0.003*"someone" + 0.003*"something" + 0.003*"company"'),
 (5,
  '0.015*"job" + 0.010*"something" + 0.008*"day" + 0.008*

In [39]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=20, wordimportance = {'tfidf':True})

In [40]:
lda_model.print_topics(num_words=15)

[(0,
  '0.003*"thats" + 0.002*"full stack developer job offer" + 0.002*"challenge" + 0.002*"quincy" + 0.002*"fcc" + 0.002*"thanks" + 0.002*"increadible website" + 0.002*"//manuelbasanta.github.io/" + 0.002*"frontend cert" + 0.002*"course amazing it´s inttoductory" + 0.002*"frontend dev" + 0.002*"cs50 edx" + 0.002*"helpful someone" + 0.002*"right aproach programming" + 0.002*"awsome course"'),
 (1,
  '0.005*"lawyer" + 0.003*"xml" + 0.003*"xsl" + 0.003*"back-end certificate" + 0.003*"second week new place" + 0.003*"first period" + 0.003*"first frontend developer job" + 0.003*"developer free code camp" + 0.003*"ideas.ataccama.com" + 0.003*"satisfied current job" + 0.002*"medium article" + 0.002*"look" + 0.002*"hey fellow camper" + 0.002*"basic" + 0.002*"thank"'),
 (2,
  '0.002*"lol" + 0.002*"web development" + 0.002*"goal" + 0.002*"agency" + 0.002*"education" + 0.001*"merry christmas" + 0.001*"schedule home day" + 0.001*"front end job" + 0.001*"wikipedia viewer" + 0.001*"=" + 0.001*"tecno

In [42]:
lsi_model.print_topics(num_words=15)

[(0,
  '-0.161*"job" + -0.138*"time" + -0.137*"company" + -0.120*"project" + -0.118*"day" + -0.118*"people" + -0.116*"resume" + -0.115*"fcc" + -0.113*"way" + -0.112*"month" + -0.106*"code" + -0.106*"year" + -0.106*"interview" + -0.094*"knowledge" + -0.093*"thing"'),
 (1,
  '-0.204*"hey fellow camper" + -0.180*"lawyer" + -0.110*"thanks" + -0.105*"hard work" + -0.104*"mern stack" + -0.100*"little bit code free time" + -0.100*"share others" + -0.100*"helpful tip" + -0.100*"whoo" + -0.100*"development knowledge gaining experience full time job" + -0.100*"many many job application" + -0.100*"full time commitment" + -0.100*"internship couple freelance project" + -0.096*"year" + -0.092*"next month"'),
 (2,
  '0.152*"code" + -0.132*"thank" + 0.117*"lawyer" + -0.113*"web development" + -0.103*"hi everyone" + 0.103*"people" + 0.101*"others" + -0.100*"august" + 0.092*"technology" + 0.091*"knowledge" + 0.087*"hey fellow camper" + -0.082*"codacademy" + -0.082*"political science" + -0.082*"good kind

### TODO

In [ ]:
############################
## raw_lda_frankjupyter2: modified to fit allrecordsPreparation2
############################
def raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=2, STOPWORDS=nltk.corpus.stopwords.words('english')):
    '''
    description: modified model based on https://www.frankcleary.com/svd/ for a more raw construction of a lda
    '''
    
   
    def metriccalc(st, normalizer, wordimportance):
        '''
        description:
        text normalization based on ALL characters in the sentence; why? Example: if two writers wrote 20 words, 2 of them very important, but one of them wrote half of characters stopwords, those 2 words wouldnt be penalized accordingly for this writer: the other wrote more important content
        '''       
        metfuncs = [
            lambda w: math.pow(0.1+float(wordimportance[w]),textbow[w]/normalizer) if w in list(wordimportance.keys()) else 0.0, #a sort of idf-normalization based on number of words in the text: the more the words in a text, the more important
            lambda w: float(wordimportance[w])*textbow[w] if w in list(wordimportance.keys()) else 0.0, #good but ignore those words with worimportance too low or 0 but that are frequent in text
            lambda w: 1.0+2.0**float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- probably the best one; because it is not normilized this indicator would simply say that if it has the word at least once is already on topic
            lambda w: float(wordimportance[w]) + textbow[w]/normalizer if w in list(wordimportance.keys()) else 0.0,
            lambda w: (1.0+textbow[w]/normalizer)*float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- apparently the second best with the adhoc word ranking
            lambda w: normalizer/textbow[w]*wordimportance[w]/sum(list(wordimportance.values())), #tfidf-ish
            lambda w: float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0
        ]
        likedict = collections.defaultdict(float)
        textbow = collections.Counter(st)
        for w in st:
            likedict[w] = metfuncs[metmodel](w)
        return likedict

    #redo_corpus_by_sts = []
    words_df = pandas.DataFrame()
    textreference = {}
    
    
    for textindex, lemmpos_r in enumerate(lemmposrecs):
        lemmpos_t = lemmpos_r[3]
        #print('lemmpos_t', len(lemmpos_t))
        lemm_sts = ''
        for lemmpos_TUPLE in lemmpos_t:
            w = lemmpos_TUPLE[2]
            lemm_sts = lemm_sts + w + ' '
        sts = lemm_sts.split('.')
        for stindex, lemmpos_st in enumerate(sts):
            treated_st, lensts = cleaningtext(lemmpos_st.split(), STOPWORDS=STOPWORDS)        
            #print('treated_st', lensts)
            if len(treated_st) > 3:
                likedict = metriccalc(treated_st, lensts, wordimportance)
                st_df = pandas.DataFrame.from_dict(likedict, orient='index')
                textindexing = str(textindex)+'_'+str(stindex)
                st_df.columns = [textindexing]
                textreference[textindexing] = {}
                textreference[textindexing]['treated_st'] = treated_st
                #st_df.columns = [str(count)]
                words_df = words_df.join(st_df, how='outer', )
    
    words_df = words_df.fillna(0)
    print("Number of unique words: %s" % len(words_df))
    print(words_df.head(10))
    #print(words_df.sort(columns=words_df.columns[0], ascending=False).head(10))
    
    return words_df, textreference